In [1]:
from __future__ import print_function

import os
from datetime import timedelta, datetime, tzinfo
import numpy as np

Define the address of the PMI server and the number of MPI workers

In [2]:
pmi_port = 'Alienware-18:54133'
partitions = 4

Prepare a list of environmental variables

In [3]:
env = []
for id in range(0, partitions):
    kvs = {
        'PMI_PORT' : pmi_port,
        'PMI_ID' : id,
    }
    env.append(kvs)

Create the rdd collection associated with the MPI workers

In [4]:
rdd = sc.parallelize(env, partitions)

Define the MPI application

In [5]:
def allreduce(kvs):
    
     os.environ["PMI_PORT"] = kvs["PMI_PORT"]
     os.environ["PMI_ID"] = str(kvs["PMI_ID"])
    
     from mpi4py import MPI
    
     comm = MPI.COMM_WORLD
     rank = comm.Get_rank()
   
     # image

     n = 2*1000000
     sendbuf = np.arange(n, dtype=np.float32)
     recvbuf = np.arange(n, dtype=np.float32)
        
     sendbuf[n-1] = 5.0;

     t1 = datetime.now()    
     comm.Allreduce(sendbuf, recvbuf, op=MPI.SUM)     
     t2 = datetime.now()
    
     out = {
        'rank' : rank,
        'time' : (t2-t1), 
        'sum'  : recvbuf[n-1]
     }
     return out

Run MPI application on Spark workers and collect the results

In [6]:
results = rdd.map(allreduce).collect()
for out in results:
    print ("rank: ", out['rank'], ", sum: ", out['sum'], ", processing time: ", out['time'])

rank:  0 , sum:  20.0 , processing time:  0:00:00.014500
rank:  1 , sum:  20.0 , processing time:  0:00:00.015380
rank:  2 , sum:  20.0 , processing time:  0:00:00.014479
rank:  3 , sum:  20.0 , processing time:  0:00:00.015245
